<a href="https://colab.research.google.com/github/qqq3964/Konkuk_NLP_Practice/blob/main/week_4_201812822%EA%B9%80%ED%83%9C%EC%99%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 자소 분리 및 결합 코드
cho_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
jung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ',
             'ㅣ']
jong_list = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ',
             'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

def jaso_split(korean_word):
    result = []
    for c in list(korean_word.strip()):
        # 현재 문자가 한글일 경우
        if '가' <= c <= '힣':
            chosung = (ord(c) - ord('가')) // 588
            jungsung = ((ord(c) - ord('가')) - (588 * chosung)) // 28
            jongsung = (ord(c) - ord('가')) - (588 * chosung) - (28 * jungsung)
            result.append((cho_list[chosung],'cho'))
            result.append((jung_list[jungsung],'jung'))
            if jongsung:
                result.append((jong_list[jongsung],'jong'))
        # 현재 문자가 한글이 아닐 경우
        else:
            result.append(c)

    return result

def jaso_combine(chosung, jungsung, jongsung=''):
    if chosung and jungsung:
        result = chr((28 * int(jung_list.index(jungsung))) + int(jong_list.index(jongsung)) + (588 * cho_list.index(chosung)) + ord('가'))
    else:
        result = ''.join([chosung, jungsung, jongsung])
    return result

In [ ]:
def tabular_parsing(jaso_list, morpheme, grammar):
    '''
    :param jaso_list: 자소 문자 리스트
    :param morpheme: 형태소 사전
    :param grammar: 품사 태그 접속 사전
    :return: 형태소 분석 결과 리스트
    '''

    # map으로 저장
    pre_jung = ''
    pre_jong = '' 
    map = []
    blue_box = []
    for unit_list in jaso_list[::-1]:
      unit,jaso = unit_list[0],unit_list[1]
      temp = []
      if jaso == 'jung':
        pre_jung = unit
        blue_box.append([])
        continue
      elif jaso == 'jong':
        temp.append(unit)
        pre_jong = unit
      else:
        temp.append(unit)
        if len(pre_jong) != 0:
          # 3가지 조건 cho,jung,jong이 모두있음
          temp.append(jaso_combine(unit,pre_jung))
          temp.append(jaso_combine(unit,pre_jung,pre_jong))
          pre_jong = ''
        elif len(pre_jong) == 0:
          temp.append(jaso_combine(unit,pre_jung))
        pre_jung = ''
      # print(f'현재 list {temp} 이전 map들 {map}')
      
      now_pos = -1

      # 사전 조사
      now_blue_box = []
      for idx,now_unit in enumerate(temp):
        if morpheme.get(now_unit):
          for i in morpheme.get(now_unit):
            # 사전검색이 성공했으므로 이를 통해 접속정보를 확인
            if abs(-idx+now_pos) > len(blue_box):
              # 마지막 정보 blue_box에서
              now_blue_box.append(now_unit+'/'+i)
              continue
            if len(blue_box[-idx+now_pos]) == 0:
              continue
            for blue_box_list in blue_box[-idx+now_pos]:
              blue_word,blue_morph = blue_box_list.split('+')[0].split('/')
              num1_check = i+'+'+blue_morph
              if num1_check in grammar:
                # 접속 검색 성공해서 여기로들어옴
                now_blue_box.append(now_unit+'/'+i+'+'+blue_box_list)
       
      # 이제 backtracking을 통한 전에 리스트들과 확인
      now_pos = -len(temp)-1
      now_unit = temp[-1]

      pre_map = []      
      for idx,check_unit in enumerate(map):
        pre_map.append(now_unit+check_unit)
        if morpheme.get(now_unit+check_unit):
          for i in morpheme.get(now_unit+check_unit):
            # 사전검색이 성공했으므로 이를 통해 접속정보를 확인
            if abs(-idx+now_pos) > len(blue_box):
              # 마지막 정보 blue_box에서
              now_blue_box.append(now_unit+check_unit+'/'+i)
              continue
            if len(blue_box[-idx+now_pos]) == 0:
              continue
            for blue_box_list in blue_box[-idx+now_pos]:
              blue_word,blue_morph = blue_box_list.split('+')[0].split('/')
              num1_check = i+'+'+blue_morph
              if num1_check in grammar:
                # 접속 검색 성공해서 여기로들어옴
                now_blue_box.append(now_unit+check_unit+'/'+i+'+'+blue_box_list)

      blue_box.append(now_blue_box)
      if jaso == 'jong':
        temp.pop()
      else:
        map = temp+pre_map

    return(blue_box[-1])

In [ ]:
if __name__ == '__main__':
    morpheme = {"건국": ["NNG"], "국": ["NNG"], "대": ["NNB"], "대가": ["NNG"], "건국대": ["NNP"], "가": ["NNG", "JKS"]}
    grammar = ["NNG+NNG", "NNB+JKS", "NNG+JKS", "NNP+NNG", "NNP+JKS"]

    # input_text = input("Input Sentence : ")
    input_text = "건국대가"
    jaso_list = jaso_split(input_text)
    print(jaso_list)
    print(tabular_parsing(jaso_list, morpheme, grammar))

[('ㄱ', 'cho'), ('ㅓ', 'jung'), ('ㄴ', 'jong'), ('ㄱ', 'cho'), ('ㅜ', 'jung'), ('ㄱ', 'jong'), ('ㄷ', 'cho'), ('ㅐ', 'jung'), ('ㄱ', 'cho'), ('ㅏ', 'jung')]
['건국/NNG+대가/NNG', '건국대/NNP+가/NNG', '건국대/NNP+가/JKS']


In [ ]:
if __name__ == '__main__':
    morpheme = {"감기": ["명사"], "감": ["동사"], "가": ["동사"], "ㅁ": ["명사형전성어미"], "기": ["명사형전성어미"], "는": ["보조사"]}
    grammar = ["명사+보조사", "동사+명사형전성어미", "명사형전성어미+명사형전성어미", "명사형전성어미+보조사"]

    # input_text = input("Input Sentence : ")
    input_text = "감기는"
    jaso_list = jaso_split(input_text)
    print(jaso_list)
    print(tabular_parsing(jaso_list, morpheme, grammar))

[('ㄱ', 'cho'), ('ㅏ', 'jung'), ('ㅁ', 'jong'), ('ㄱ', 'cho'), ('ㅣ', 'jung'), ('ㄴ', 'cho'), ('ㅡ', 'jung'), ('ㄴ', 'jong')]
['가/동사+ㅁ/명사형전성어미+기/명사형전성어미+는/보조사', '감/동사+기/명사형전성어미+는/보조사', '감기/명사+는/보조사']
